In [1]:
import scanpy as sc
import anndata2ri
import os
import logging
import rpy2
import rpy2.rinterface_lib.callbacks as rcb
import rpy2.robjects as ro

RHOME = "/share/anaconda3/envs/SpaBench/lib/R"
RLIB = os.path.join(RHOME,'library')
r = ro.r
r(f'Sys.setenv(R_HOME = \"{RHOME}\")')
r(f'.libPaths(\"{RLIB}\")')
rcb.logger.setLevel(logging.ERROR)

ro.pandas2ri.activate()
anndata2ri.activate()

%load_ext rpy2.ipython

/tmp/ipykernel_16571/1070530140.py:17: DeprecationWarning: The global conversion available with activate() is deprecated and will be removed in the next major release. Use a local converter.
  anndata2ri.activate()


In [9]:
%%R -o run_bs_impute -o run_bs_base
library(SingleCellExperiment)
library(scuttle)
library(scran)
library(scater)
library(BiocSingular)
library(sceasy)
library(Seurat)
library(BayesSpace)

run_bs_base <- function(sce, save_dir, q=NULL){
    df = colData(sce)
    df = cbind(df,reducedDim(sce))
    colnames(df) = c(colnames(df)[1:3],'col','row')
    sce@colData = df
    
    sce.p = spatialPreprocess(sce, platform="Visium")
    cat('Clustering','\n')
    if (is.null(q)){
        q=length(unique(sce@colData$layer_guess_reordered))
    }
    sce@colData = df
    sce.p = spatialCluster(sce.p, q=q, d=15, platform='Visium', gamma=3)
    
    res = as.data.frame(sce.p$spatial.cluster)
    row.names(res) = colnames(sce)
    write.csv(res,save_dir)
}

run_bs_impute <- function(sce, save_dir, use_rep='SPANVE.IMPUTED.PCA.PRE', q = NULL){
    df = colData(sce)
    df = cbind(df,reducedDim(sce))
    colnames(df) = c(colnames(df)[1:3],'col','row')
    sce@colData = df
    if (is.null(q)){
        q=length(unique(sce@colData$ground_truth))
    }
    cat('Clustering','\n')
    sce.i <- spatialCluster(sce, use.dimred=use_rep, q=q, d=15, platform='Visium', gamma=3)
    # sce.p = spatialPreprocess(sce, platform="Visium")
    # cat('Clustering','\n')
    # sce.p = spatialCluster(sce.p, q=length(unique(sce@colData$ground_truth)), d=15, platform='Visium', gamma=3)

    res = as.data.frame(sce.i$spatial.cluster)
    row.names(res) = colnames(sce)
    write.csv(res,save_dir)
}

## SpatialLIBD

In [3]:
adata = sc.read_h5ad('./recode/IMPUTEDLIBD/data/151507.h5ad')
adata

AnnData object with n_obs × n_vars = 4226 × 15818
    obs: 'n_genes', 'ground_truth'
    var: 'n_cells', 'highly_variable', 'means', 'dispersions', 'dispersions_norm', 'n_counts'
    uns: 'hvg', 'pca'
    obsm: 'X_pca', 'cell_ranger.pca', 'moran.pca', 'p.spanve-k.imputated.pca', 'p.spanve-k.pca', 'spanve-k.imputated.pca', 'spanve-k.pca', 'spatial', 'spatialde.pca'
    varm: 'PCs'
    layers: 'counts'

In [ ]:
import time
data_dir = './recode/IMPUTEDLIBD/data/'
result_dir = './recode/IMPUTEDLIBD/BS/'

for file in os.listdir(data_dir):
    print(time.ctime(),file)
    file_dir = os.path.join(data_dir, file)
    adata = sc.read_h5ad(file_dir)
    adata.layers['counts'] = adata.X.copy()
    for use_rep in ['spanve-k.pca', 'p.spanve-k.pca', 'spanve-k.imputated.pca']:
        save_file_dir = os.path.join(result_dir,f'{use_rep}_{file.split(".")[0]}.csv')
        if os.path.exists(save_file_dir): continue
        print(time.ctime(), use_rep)
        run_bs_impute(adata, save_file_dir, use_rep)

Tue Feb 20 14:41:11 2024 151508.h5ad
Tue Feb 20 14:41:11 2024 spanve-k.imputated.pca
Clustering 
Tue Feb 20 15:35:03 2024 151509.h5ad
Tue Feb 20 15:35:03 2024 spanve-k.imputated.pca
Clustering 
Tue Feb 20 16:35:06 2024 151510.h5ad
Tue Feb 20 16:35:07 2024 spanve-k.imputated.pca
Clustering 
Tue Feb 20 17:42:03 2024 151670.h5ad
Tue Feb 20 17:42:04 2024 spanve-k.imputated.pca
Clustering 
Tue Feb 20 18:25:22 2024 151671.h5ad
Tue Feb 20 18:25:23 2024 spanve-k.imputated.pca
Clustering 
Tue Feb 20 19:16:27 2024 151672.h5ad
Tue Feb 20 19:16:27 2024 spanve-k.pca
Clustering 
Tue Feb 20 20:12:19 2024 p.spanve-k.pca
Clustering 
Tue Feb 20 21:02:03 2024 spanve-k.imputated.pca
Clustering 
Tue Feb 20 21:52:17 2024 151674.h5ad
Tue Feb 20 21:52:17 2024 spanve-k.pca
Clustering 
Tue Feb 20 22:56:05 2024 p.spanve-k.pca
Clustering 
Tue Feb 20 23:52:15 2024 spanve-k.imputated.pca
Clustering 
Wed Feb 21 01:44:55 2024 151675.h5ad
Wed Feb 21 01:44:55 2024 spanve-k.pca
Clustering 
Wed Feb 21 02:29:17 2024 p.spa

## Invasive indutal Breast Cancer

In [10]:
import pandas as pd
import numpy as np
import scanpy as sc
import os
# import sklearn
# import matplotlib.pyplot as plt
# import seaborn as sns
# import squidpy as sq
# import sys
# sys.path.append('./Softs/')
# import Spanve_beta
# print(Spanve_beta.__version__)
# from Spanve_beta import *

# svmodel = Spanve(adata)
# svmodel.load('./data/Spanve_Human_breas_cancer.csv')
# adata.obsm['normalized_pca'] = sc.pp.pca(adata.layers['normalized'])
# adata.obsm['imputated_pca'] = sc.pp.pca(adata[:, adata.var.query('spanve_spatial_features').index].layers['imputated'])

In [ ]:
adata = sc.datasets.visium_sge('Visium_Human_Breast_Cancer')
sc.pp.filter_genes(adata, min_cells=10)
adata.var_names_make_unique()
adata.layers['counts'] = adata.X.copy()
run_bs_base(adata, './recode/BreastCancer/base_BayesSpace.csv', q = 7)
result_dir = './recode/BreastCancer/'

for use_rep in ['normalized_pca', 'imputated_pca']:
    print(time.ctime(),use_rep)
    save_file_dir = os.path.join(result_dir,f'{use_rep}_BayesSpace.csv')
    if os.path.exists(save_file_dir): continue
    run_bs_impute(adata, save_file_dir, use_rep, q = 8)